## We will train and Store the Model which can be used to generate the data

In [3]:
import pandas as pd
from ctgan import CTGAN

In [ ]:
#!pip install fastparquet

In [31]:
# Incase you want to change type of a column add it as schema
custom_schema = {'age' : 'int64' ,'sex' : 'object' ,'bmi' : 'float64' ,'children' : 'int64' ,'smoker' : 'object' ,'region' : 'object' ,'charges' : 'float64'}

df = pd.read_csv('insurance.csv',dtype = custom_schema)
df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [32]:
# Incase you want to see pandas has evaluate wrong datatype
for i in list(df.columns.values):
    print("'{}' : '{}' ,".format(i, (df[i].dtype)))

'age' : 'int64' ,
'sex' : 'object' ,
'bmi' : 'float64' ,
'children' : 'int64' ,
'smoker' : 'object' ,
'region' : 'object' ,
'charges' : 'float64' ,


In [33]:
#Strip any space leading or trailing
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

print(df.head(2))

   age     sex    bmi  children smoker     region     charges
0   19  female  27.90         0    yes  southwest  16884.9240
1   18    male  33.77         1     no  southeast   1725.5523


In [34]:
def getSyntheticData(df,sample_size,modelSave):
    ctgan = CTGAN(verbose=True)
    ctgan.fit(df, list(df.columns.values),epochs= 200)

    ## Synthetic Data Generation
    ## It will take time. Don't create more than i Million at a time.

    samples = ctgan.sample(sample_size)
    
    ## Save the sample as csv or parqute
    samples.to_csv('testdata.csv',encoding='utf-8',index=False,header=True)
    samples.to_parquet('testdata1.parquet',engine='fastparquet')
    
    #If wanted to save model for later use
    if(modelSave):
        ctgan.save('ctagn_insurance_model.pkl')
    
    samples.head(10)

In [35]:
getSyntheticData(df,10,True)

Gen. (2.05) | Discrim. (-0.03): 100%|██████████| 200/200 [02:30<00:00,  1.33it/s]


## Now We can get samples directly from our trained model

In [37]:
from ctgan import CTGAN

model_path = 'ctagn_insurance_model.pkl'

ctgan = CTGAN.load(model_path)
# Generate Synthetic samples

sythetic_data = ctgan.sample(100)
sythetic_data.to_parquet('testdata2.parquet',engine='fastparquet')

sythetic_data.head(100)

d:\github\synthetic-data-ctgan\.venv\Lib\site-packages\ctgan\synthesizers\base.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path)


,age,sex,bmi,children,smoker,region,charges
0,48,male,23.600,0,yes,southeast,9447.38240
1,61,female,31.065,4,no,southeast,4877.98105
2,37,male,39.490,3,yes,southeast,14474.67500
3,20,male,32.200,3,no,northwest,40941.28540
4,52,male,29.700,3,no,northeast,7441.50100
...,...,...,...,...,...,...,...
95,59,male,28.310,0,no,southeast,14478.33015
96,26,female,35.750,2,no,southeast,2103.08000
97,45,male,31.445,0,no,northeast,14711.74380
98,24,male,28.700,3,yes,northeast,13430.26500


## We can define these params like this as well. Need to test it later

field_types = {"MARITAL":{"type":"categorical"},"RACE":{"type":"categorical"},"ETHNICITY":{"type":"categorical"},"GENDER":{"type":"categorical"},"PREFIX":{"type":"categorical"},"SUFFIX":{"type":"categorical"},"MAIDEN":{"type":"categorical"},"BIRTHPLACE":{"type":"categorical"},"CITY":{"type":"categorical"},"STATE":{"type":"categorical"},"COUNTY":{"type":"categorical"},"ZIP":{"type":"categorical"},"LAT":{"type":"categorical"},"LON":{"type":"categorical"},"HEALTHCARE_EXPENSES":{"type":"numerical","subtype":"float"},"HEALTHCARE_COVERAGE":{"type":"numerical","subtype":"float"},"BIRTHDATE":{"type":"datetime","format":"%m/%d-%Y"},"DEATHDATE":{"type":"datetime","format":"%m/%d-%Y"}}

field_transformers = {"MARITAL":"categorical","RACE":"categorical","ETHNICITY":"categorical","GENDER":"categorical","PREFIX":"categorical","SUFFIX":"categorical","MAIDEN":"categorical","BIRTHPLACE":"categorical","CITY":"categorical","STATE":"categorical","COUNTY":"categorical","ZIP":"categorical","HEALTHCARE_EXPENSES":"float","HEALTHCARE_COVERAGE":"float","BIRTHDATE":"datetime","DEATHDATE":"datetime"}

{"SSN":"ssn","ADDRESS":"address","LAT":"latitude","LON":"longitude","FIRST":"first_name","LAST":"last_name","Id":"uuid4"}


model = CTGAN(field_types=field_types, field_transformers=field_transformers, anonymize_fields=anonymize, epochs=1) 
